In [6]:
import json, requests
from bs4 import BeautifulSoup

import feedparser
from markdownify import markdownify

In [19]:
rss = feedparser.parse("https://www.nature.com/nature.rss")
rss["entries"]

[{'id': 'https://www.nature.com/articles/d41586-023-00776-w',
  'title': 'Bird-flu virus makes itself at home in Canada’s foxes and skunks',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://www.nature.com/nature.rss?error=cookies_not_supported&code=b0ce6e2f-4472-46ce-9281-a67addc94f76',
   'value': 'Bird-flu virus makes itself at home in Canada’s foxes and skunks'},
  'links': [{'rel': 'alternate',
    'type': 'text/html',
    'href': 'https://www.nature.com/articles/d41586-023-00776-w'}],
  'link': 'https://www.nature.com/articles/d41586-023-00776-w',
  'content': [{'type': 'text/html',
    'language': None,
    'base': 'https://www.nature.com/nature.rss?error=cookies_not_supported&code=b0ce6e2f-4472-46ce-9281-a67addc94f76',
    'value': '<p>Nature, Published online: 17 March 2023; <a href="https://www.nature.com/articles/d41586-023-00776-w">doi:10.1038/d41586-023-00776-w</a></p>The virulent H5N1 strain now sweeping across the world is adapting to its 

In [4]:
rss["entries"][0]

{'id': 'https://www.nature.com/articles/d41586-023-00776-w',
 'title': 'Bird-flu virus makes itself at home in Canada’s foxes and skunks',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://www.nature.com/nature.rss?error=cookies_not_supported&code=c2d858cb-5a48-4a54-a6f1-f0d810b02a22',
  'value': 'Bird-flu virus makes itself at home in Canada’s foxes and skunks'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://www.nature.com/articles/d41586-023-00776-w'}],
 'link': 'https://www.nature.com/articles/d41586-023-00776-w',
 'content': [{'type': 'text/html',
   'language': None,
   'base': 'https://www.nature.com/nature.rss?error=cookies_not_supported&code=c2d858cb-5a48-4a54-a6f1-f0d810b02a22',
   'value': '<p>Nature, Published online: 17 March 2023; <a href="https://www.nature.com/articles/d41586-023-00776-w">doi:10.1038/d41586-023-00776-w</a></p>The virulent H5N1 strain now sweeping across the world is adapting to its mammalian host

In [3]:
len(rss["entries"])

75

In [14]:
def article_content(soup : BeautifulSoup) -> str:
    
    
    if soup.find_all("div", {"class": "c-article-body main-content"}):
        
    
    return None

In [ ]:
def article_content(soup : BeautifulSoup) -> str:
    
    title = soup.find("h1", {"class": "c-article-magazine-title"}).text.strip()
    field = soup.find("li", {"class":"c-article-subjects__item"}).text.strip()

In [18]:
r = requests.request("GET","https://www.nature.com/articles/d41586-023-00818-3")
soup = BeautifulSoup(r.content,"html.parser")

In [24]:
soup.find("li", {"class":"c-article-subjects__item"}).text.strip()

'Molecular biology'

In [3]:
print(subscribe_required(soup))

NameError: name 'subscribe_required' is not defined

In [9]:
html = soup.find("div", {"class": "c-article-body main-content"})
h = markdownify(html, heading_style="ATX")

TypeError: 'NoneType' object is not callable

In [10]:
html

<div class="c-article-body main-content">
<figure class="figure">
<picture class="embed intensity--high">
<source sizes="(max-width: 319px) 319px, (min-width: 1023px) 100vw,  767px" srcset="//media.nature.com/lw767/magazine-assets/d41586-023-00818-3/d41586-023-00818-3_24631076.jpg?as=webp 767w, //media.nature.com/lw319/magazine-assets/d41586-023-00818-3/d41586-023-00818-3_24631076.jpg?as=webp 319w" type="image/webp"/>
<img alt="" class="figure__image" loading="lazy" src="//media.nature.com/lw767/magazine-assets/d41586-023-00818-3/d41586-023-00818-3_24631076.jpg"/>
<figcaption>
<p class="figure__caption u-sans-serif"><span class="mr10">The OR51E2 receptor is activated by propionate, which has a cheesy odour.</span><span>CREDIT: Antonio Nardelli/ EyeEm/ Getty Images</span></p>
</figcaption>
</picture>
</figure><p>It is thanks to proteins in the nose called odour receptors that we find the smell of roses pleasant and that of rotting food foul. But little is known about how these receptors

In [17]:
html[0]

'I'

In [16]:
html = markdownify(html[0])

print(html)

It is thanks to proteins in the nose called odour receptors that we find the smell of roses pleasant and that of rotting food foul. But little is known about how these receptors detect molecules and translate them into scents.

Now, for the first time, researchers have mapped the precise 3D structure of a human odour receptor, taking a step forwards in understanding the most enigmatic of our senses.

The study, published in *Nature* on 15 March[1](#ref-CR1), describes an olfactory receptor called OR51E2 and shows how it ‘recognizes’ the smell of cheese through specific molecular interactions that switch the receptor on.

“It’s basically our first picture of any odour molecule interacting with one of our odour receptors,” says study co-author Aashish Manglik, a pharmaceutical chemist at the University of California, San Francisco.

Smell mystery
-------------

The human genome contains genes encoding 400 olfactory receptors that can detect many odours. Mammalian odour-receptor genes wer

In [14]:
import re
html = re.findall("</figure>(.*?)</div>", str(html), re.DOTALL)